# Prepare Train Data

In [2]:
from datetime import datetime, timezone, date
from datetime import timedelta
from pathlib import Path
import pandas as pd
import pyarrow as pa

In [3]:
CONSUMPTION_DATA_PATH = Path("../data/consumption")
CONSUMPTION_NE5_FILE = CONSUMPTION_DATA_PATH / "NE5_Export.csv"
CONSUMPTION_NE7_FILE = CONSUMPTION_DATA_PATH / "NE7_Export.csv"
TRAIN_DATA_PATH = Path('../data/consumption/final_train.parquet')
DATA_PATH_2022 = Path('../data/consumption/2022.parquet')

METEO_PATH = Path("../data/meteoswiss/reh_nzz.csv")
METEO_STATION = "REH"
METEO_TEMP_PARAMETER = "tre200h0"

TIME_FORMAT = "dd.MM.yyyy"
WINDOW_DAYS = 1
START_DATE = date(2010, 1, 1)
END_DATE = date(2022, 1, 1)

In [4]:
#ne5File = spark.read.csv(CONSUMPTION_NE5_FILE, header=True, inferSchema=True, sep=";")
#ne7File = spark.read.csv(CONSUMPTION_NE7_FILE, header=True, inferSchema=True, sep=";")

# consumption = (ne5File.alias("ne5")
#                       .join(ne7File.alias("ne7"), "Date")
#                       .withColumn("NE5Consumption", f.expr("ne5.Value"))
#                       .withColumn("NE7Consumption", f.expr("ne7.Value"))
#                       .withColumn("Date", f.to_date(f.col("Date"), TIME_FORMAT))
#                       .filter((f.col("Date") >= START_DATE) & (f.col("Date") < END_DATE))
#                       .select("Date", "NE5Consumption", "NE7Consumption")
#               )

consumption = pd.read_csv('https://data.stadt-zuerich.ch/dataset/ewz_stromabgabe_netzebenen_stadt_zuerich/download/ewz_stromabgabe_netzebenen_stadt_zuerich.csv')
consumption['Date'] = pd.to_datetime(consumption['Timestamp'], errors='raise', utc=True)
consumption['Date'] = consumption['Date'].dt.tz_convert('Europe/Berlin')
consumption.rename(columns={'Value_NE5': 'NE5Consumption', 'Value_NE7': 'NE7Consumption'}, inplace=True)
consumption = consumption.groupby(consumption.Date.dt.date).sum(numeric_only=True)

consumption.tail()

,NE5Consumption,NE7Consumption
Date,,
2022-10-29,2.072831e+06,4.298467e+06
2022-10-30,1.995849e+06,4.206186e+06
2022-10-31,2.466076e+06,4.864538e+06
2022-11-01,2.491564e+06,5.000871e+06
2022-11-02,2.149638e+04,3.566598e+04


### Station REH

In [5]:
"""
meteoData = (spark.read.format("delta")
                         .load("/MeteoSwiss/Measurement/Delta")
                         .filter((f.col("Station") == METEO_STATION) & (f.col("Parameter") == METEO_TEMP_PARAMETER))
                         .withColumn("Date", f.to_date(f.col("TimestampUtc")))
                         .groupBy("Date")
                         .agg(f.avg(f.col("Value")).alias("Temperature"))
            )
"""

meteoData = pd.read_csv(METEO_PATH, encoding='iso-8859-1', sep=';')
meteoData = meteoData[meteoData.abbr == METEO_STATION]
meteoData['Date'] = pd.to_datetime(meteoData['time'], format='%Y%m%d%H%M', utc=True)

# Convert Timezone!
#meteoData['Date'] = meteoData['Date'].dt.tz_convert('Europe/Berlin')

meteoData = meteoData.groupby(meteoData.Date.dt.date).agg(Temperature = (METEO_TEMP_PARAMETER, 'mean'))

meteoData.tail()

,Temperature
Date,
2022-10-27,12.500000
2022-10-28,12.583333
2022-10-29,13.066667
2022-10-30,11.270833
2022-10-31,10.400000


### Station SMA

In [6]:
"""
meteoData = pd.concat([
    pd.read_csv('https://data.geo.admin.ch/ch.meteoschweiz.klima/nbcn-tageswerte/nbcn-daily_SMA_previous.csv', encoding='utf-8', sep=';'),
    pd.read_csv('https://data.geo.admin.ch/ch.meteoschweiz.klima/nbcn-tageswerte/nbcn-daily_SMA_current.csv', encoding='utf-8', sep=';')
])
meteoData['Date'] = pd.to_datetime(meteoData['date'], format='%Y%m%d')
meteoData['Date'] = meteoData['Date'].dt.date
meteoData = meteoData[meteoData.Date >= date(2009, 1, 1)]
meteoData.rename(columns={'tre200d0': 'Temperature'}, inplace=True)
meteoData = meteoData[['Date', 'Temperature']]
meteoData = meteoData.set_index('Date')

meteoData.head()
"""

"\nmeteoData = pd.concat([\n    pd.read_csv('https://data.geo.admin.ch/ch.meteoschweiz.klima/nbcn-tageswerte/nbcn-daily_SMA_previous.csv', encoding='utf-8', sep=';'),\n    pd.read_csv('https://data.geo.admin.ch/ch.meteoschweiz.klima/nbcn-tageswerte/nbcn-daily_SMA_current.csv', encoding='utf-8', sep=';')\n])\nmeteoData['Date'] = pd.to_datetime(meteoData['date'], format='%Y%m%d')\nmeteoData['Date'] = meteoData['Date'].dt.date\nmeteoData = meteoData[meteoData.Date >= date(2009, 1, 1)]\nmeteoData.rename(columns={'tre200d0': 'Temperature'}, inplace=True)\nmeteoData = meteoData[['Date', 'Temperature']]\nmeteoData = meteoData.set_index('Date')\n\nmeteoData.head()\n"

## Export

In [7]:
data = consumption.join(meteoData, "Date").reset_index(drop=False)

In [8]:
# data.write.parquet("/ConsumptionModel/Data/final_train.parquet")
data[(data.Date >= START_DATE) & (data.Date < END_DATE)].to_parquet(TRAIN_DATA_PATH, engine='pyarrow')

# 2022
data[(data.Date >= date(2021, 12, 26)) & (data.Date < date(2022, 10, 31))].to_parquet(DATA_PATH_2022, engine='pyarrow')
